# How to set up workflows with Kili

In this tutorial, we'll learn how to set up basic Kili workflows:

1. [Managing reviews](#-Managing-reviews)
    1. [Placing a specific percentage of project assets in the review queue](#-Placing-a-specific-percentage-of-project-assets-in-the-review-queue)
    1. [Placing specific assets in the review queue](#-Placing-specific-assets-in-the-review-queue)
    1. [Sending an asset back to the labeling queue](#-Sending-an-asset-back-to-the-labeling-queue)
1. [Setting up consensus](#-Setting-up-consensus)
    1. [Assigning a specific percentage of project assets for consensus](#-Assigning-a-specific-percentage-of-project-assets-for-consensus)
    1. [Assigning specific assets for computing consensus KPIs](#-Assigning-specific-assets-for-computing-consensus-KPIs)
1. [Setting up honeypot](#-Setting-up-honeypot)
1. [Assigning labelers to assets](#-Assigning-labelers-to-assets)
1. [Prioritizing assets in the labeling queue](#-Prioritizing-assets-in-the-labeling-queue)

To work with this notebook, you'll have to install and instantiate Kili.

In [ ]:
!pip install --upgrade kili
from kili.client import Kili
import getpass

In [ ]:
KILI_API_KEY = getpass.getpass("Please enter your API key: ")

In [ ]:
kili = Kili(
    api_key=KILI_API_KEY,
    # api_endpoint="https://cloud.kili-technology.com/api/label/v2/graphql", 
    # the line above can be uncommented and changed if you are working with an on-premise version of Kili
)

In [ ]:
project_id = <TYPE_YOUR_PROJECT_ID_HERE>

For information on how to set up a Kili project, refer to the `basic_project_setup.ipynb` tutorial.

## Managing reviews

### Placing a specific percentage of project assets in the review queue 

You can set up the percentage of assets that will automatically appear in the review queue (1-100%).
In our example, we'll assign 50% of all project assets for review.

In [ ]:
kili.update_properties_in_project(project_id=project_id, review_coverage=50)

Just to make sure that we got it right, let's check on our project's review coverage value:

In [ ]:
review_coverage = kili.projects(project_id=project_id, fields=['reviewCoverage'])
print(f'The review coverage set up for project id {project_id} is {review_coverage[0]["reviewCoverage"]}%.')

### Placing specific assets in the review queue

You can choose to set specific assets to be placed in the review queue. First, let's access the list your available assets:

In [ ]:
kili.assets(project_id=project_id)

IMPORTANT:
- Use Kili's internal asset IDs ('id' field)
- Only labeled assets may be assigned for review ('status': 'LABELED'})

For example purposes, we'll simulate adding some labels to one of the assets. To make things simple, we'll use the first asset:

In [ ]:
asset_id = kili.assets(project_id=project_id)[0]['id']
print(f'Selected asset ID is: {asset_id}.')

Now, let's append a simple label to the asset:

In [ ]:
kili.append_labels(asset_id_array=[asset_id], json_response_array=[{'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}], label_type='DEFAULT')

Note that we 're using lists here, so if you want to add labels to more than one asset, you simply need to add more items to the list. For example:

```
kili.append_labels(asset_id_array=[asset_id, asset_id2], json_response_array=[{'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}, {'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_A'}]}}], label_type='DEFAULT')
```

Let's check the newly-added label:

In [ ]:
kili.assets(project_id=project_id, asset_id=asset_id, fields=['labels.jsonResponse'])

We've added a label so we can now set the asset for review.

In [ ]:
kili.add_to_review([asset_id])

Again, if you'd like to set more assets for review, simply add more items to the list:

```
kili.add_to_review([asset_id, asset_id2])
```

Let's make sure we've done it right:

In [ ]:
asset_status = kili.assets(project_id=project_id, asset_id=asset_id, fields=['status'])
print(f'The status for asset ID {project_id} is {asset_status[0]["status"]}.')

For more information on asset statuses, refer to our [documentation](https://docs.kili-technology.com/docs/asset-lifecycle).

### Sending an asset back to the labeling queue

You can send a specific labeled asset back to the labeling queue.
First, let's access the list of your available assets:

In [ ]:
kili.assets(project_id=project_id)

For example purposes, we'll select the first asset:

In [ ]:
asset_id = kili.assets(project_id=project_id)[0]['id']
print(f'Selected asset ID is: {asset_id}.')

Let's simulate adding a label to the asset:

In [ ]:
kili.append_labels(asset_id_array=[asset_id], json_response_array=[{'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}], label_type='DEFAULT')

Note that we 're using lists here, so if you want to add labels to more than one asset, you simply need to add more items to the list. For example:

```
kili.append_labels(asset_id_array=[asset_id, asset_id2], json_response_array=[{'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}, {'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_A'}]}}], label_type='DEFAULT')
```

... and make sure we got it right:

In [ ]:
kili.assets(project_id=project_id, asset_id=asset_id, fields=['labels.jsonResponse'])

Now, we'll send this asset back to the labeling queue:

In [ ]:
kili.send_back_to_queue(asset_ids=[asset_id])

Again, if you need to, you can send more assets at once:

```
kili.send_back_to_queue(asset_ids=[asset_id, asset_id2])
```

The new status for the asset should be "ONGOING". 

In [ ]:
asset_status = kili.assets(project_id=project_id, asset_id=asset_id, fields=['status'])
print(f'The status for asset ID {asset_id} is {asset_status[0]["status"]}.')

For more information on asset statuses, refer to our [documentation](https://docs.kili-technology.com/docs/asset-lifecycle).

## Setting up consensus

Consensus works by having more than one labeler annotate the same asset. When the asset is labeled, a consensus score is calculated to measure the agreement level between the different annotations for a given asset. This is a key measure for controlling label production quality.

To set up consensus, you'll need to have at least two project members.
Let's check how many users we currently have in our project:

In [ ]:
kili.count_project_users(project_id=project_id)

For information on how to add users and assign them to your project, refer to the `basic_project_setup.ipynb` tutorial.

### Assigning a specific percentage of project assets for consensus

Let's assign the total percentage of all assets that will have to be labeled by more than one reviewer. To make things simple, we'll set consensus to all of the assets (100%):

In [ ]:
kili.update_properties_in_project(project_id=project_id, consensus_tot_coverage=100)

Just to be sure, we'll verify this:

In [ ]:
cons_tot_coverage = kili.projects(project_id=project_id, fields=["consensusTotCoverage"])
print(f'The consensus coverage for project ID {project_id} is {cons_tot_coverage[0]["consensusTotCoverage"]}%.')

### Assigning specific assets for computing consensus KPIs

You can manually select specific project assets to be used for computing consensus KPIs. First, let's access the list of your available assets:

In [ ]:
kili.assets(project_id=project_id)

For example purposes, we'll select the first asset:

In [ ]:
asset_id = kili.assets(project_id=project_id)[0]['id']
print(f'Selected asset ID is: {asset_id}.')

Now, let's assign this asset for consensus KPIs:

In [ ]:
kili.update_properties_in_assets(asset_ids=[asset_id], is_used_for_consensus_array=[True])

Note that you can operate on more than one asset at once. For example:

```
kili.update_properties_in_assets(asset_ids=[asset_id, asset_id2], is_used_for_consensus_array=[True, False])
```

Let's check if everything's correct:

In [ ]:
is_used_for_consensus = kili.assets(project_id=project_id, asset_id=asset_id, fields=["isUsedForConsensus"])
print(f'Asset ID: {asset_id} is used for consensus: {is_used_for_consensus[0]["isUsedForConsensus"]}.')

For more information on consensus, refer to our [documentation](https://docs.kili-technology.com/docs/consensus-overview).

## Setting up honeypot

Honeypot (or __gold standard__) is a tool for auditing the work of labelers by measuring the accuracy of their annotations.
Honeypot works by interspersing assets with defined ground truth label in the annotation queue. This way you can measure the agreement level between your ground truth and the annotations made by labelers.

You can manually select specific project assets to be used as honeypots.
First, let's access the list of your available assets:

In [ ]:
kili.assets(project_id=project_id)

For example purposes, we'll select the first asset:

In [ ]:
asset_id = kili.assets(project_id=project_id)[0]['id']
print(f'Selected asset ID is: {asset_id}.')

Now, let's assign this asset as a honeypot:

In [ ]:
kili.update_properties_in_assets(asset_ids=[asset_id], is_honeypot_array=[True])

Note that you can assign more than one asset at once. For example:

```
kili.update_properties_in_assets(asset_ids=[asset_id, asset_id2], is_honeypot_array=[True, False])
```

... and now let's verify:

In [ ]:
is_used_as_honeypot = kili.assets(project_id=project_id, asset_id=asset_id, fields=["isHoneypot"])
print(f'Asset ID: {asset_id} is used as honeypot: {is_used_as_honeypot[0]["isHoneypot"]}.')

For more information on honeypot, refer to our [documentation](https://docs.kili-technology.com/docs/consensus-overview).

## Assigning labelers to assets

You can assign specific labelers to specific assets in your project.
First, let's find out what the IDs of our users are:

In [ ]:
all_users = kili.project_users(project_id=project_id)
print(all_users)

For information on how to add users and assign them to your project, refer to the `basic_project_setup.ipynb` tutorial.

For example purposes, we'll select our first user:

In [ ]:
user_email = all_users[0]['user']['email']
print(f"Our selected user's email is: {user_email}")

Now, we'll select the first asset from a list of available assets:

In [ ]:
kili.assets(project_id=project_id)
asset_id = kili.assets(project_id=project_id)[0]['id']
print(f'Selected asset ID is: {asset_id}.')

The last thing to do is to assign the user email to the selected asset ID:

In [ ]:
kili.update_properties_in_assets(asset_ids=[asset_id], to_be_labeled_by_array=[[user_email]])

Note that you can update more than one asset at once. For example:

```
kili.update_properties_in_assets(asset_ids=[asset_id, asset_id2], to_be_labeled_by_array=[[user_email, user_email2]])
```

Let's verify that:

In [ ]:
to_be_labeled_by = kili.assets(project_id=project_id, asset_id=asset_id, fields=["isToBeLabeledBy"])
print(f'Asset ID: {asset_id} is is set to be labeled by: {to_be_labeled_by[0]["isToBeLabeledBy"]}.')

For information on assigning assets to users, refer to our [documentation](https://docs.kili-technology.com/docs/queue-prioritization).

## Prioritizing assets in the labeling queue

If you have certain assets that you need to have labeled earlier than the rest, you can use Kili's asset prioritization methods.
For example purposes, we'll select the first asset in our project:

In [ ]:
kili.assets(project_id=project_id)
asset_id = kili.assets(project_id=project_id)[0]['id']
print(f'Selected asset ID is: {asset_id}.')

Now, all that's left to do is to set specific priorities to assets:

In [ ]:
kili.update_properties_in_assets(asset_ids=[asset_id], priorities=[1])

For your convenience, you can update priorities for more than one asset at once. For example:

```
kili.update_properties_in_assets(asset_ids=[asset_id, asst_id2], priorities=[1, 10])
```

Let's verify that:

In [ ]:
priority = kili.assets(project_id=project_id, asset_id=asset_id, fields=["priority"])
print(f'Asset ID: {asset_id} is is set as priority: {priority[0]["priority"]}.')

For information on setting asset priorities, refer to our [documentation](https://docs.kili-technology.com/docs/queue-prioritization).

## Summary

Done. We've learned how to handle the review workflow, set up consensus and honeypot in a project, assign specific labelers to specific assets, and how to prioritize assets in the labeling queue. Well done!